In [9]:
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
sys.path.insert(0, project_root)

print(f"Added {project_root} to Python path")



Added /rsrch5/home/plm/yshokrollahi/vitamin-p-new to Python path


In [23]:
# Now try the imports again
from models.model import CellSwin
from data.dataset import CellSegmentationDataset
from models.metrics import iou_score, calculate_object_based_metrics
import torch
from torchvision import transforms
from data.data_loading import load_all_folds, create_train_val_test_split

In [24]:
import torch
import os

# Get the project root directory
project_root = os.path.dirname(os.getcwd())  # Assuming we're in a notebook in a subdirectory

# Path to your saved model
model_path = os.path.join(project_root, 'checkpoints', 'v3_fold2.pth')

# Print the model path to verify
print(f"Loading model from: {model_path}")

# Determine the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model architecture
trained_model = CellSwin().float()

# Load the saved state dict
state_dict = torch.load(model_path, map_location=device)

# Load the state dict into your model
trained_model.load_state_dict(state_dict)

# Move the model to the appropriate device
trained_model = trained_model.to(device)

# Set the model to evaluation mode
trained_model.eval()

print(f"Model loaded successfully and moved to {device}!")

Loading model from: /rsrch5/home/plm/yshokrollahi/vitamin-p-new/checkpoints/v3_fold2.pth
Model loaded successfully and moved to cuda!


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
import random

def visualize_prediction_with_metrics(model, test_dataset, device, num_samples=3):
    model.eval()
    
    # Randomly select indices
    indices = random.sample(range(len(test_dataset)), num_samples)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    overall_f1 = 0
    overall_precision = 0
    overall_recall = 0
    
    for i, idx in enumerate(indices):
        image, mask = test_dataset[idx]
        image = image.unsqueeze(0).to(device)  # Add batch dimension
        
        with torch.no_grad():
            output = model(image)
        
        image = image.cpu().numpy().transpose(0, 2, 3, 1).squeeze()
        mask = mask.cpu().numpy().squeeze()
        output = output.cpu().numpy().squeeze()
        
        # Calculate metrics
        pred_mask = (output > 0.5).astype(np.uint8)
        f1 = f1_score(mask.flatten(), pred_mask.flatten(), average='binary')
        precision = precision_score(mask.flatten(), pred_mask.flatten(), average='binary')
        recall = recall_score(mask.flatten(), pred_mask.flatten(), average='binary')
        
        overall_f1 += f1
        overall_precision += precision
        overall_recall += recall
        
        # Plotting
        axes[i, 0].imshow(image)
        axes[i, 0].set_title(f"Input Image {idx}")
        axes[i, 0].axis('off')
        
        # Use a custom colormap for true mask
        axes[i, 1].imshow(mask, cmap='gray_r', vmin=0, vmax=1)
        axes[i, 1].set_title(f"True Mask {idx}")
        axes[i, 1].axis('off')
        
        # Use a custom colormap for predicted mask
        axes[i, 2].imshow(pred_mask, cmap='gray_r', vmin=0, vmax=1)
        axes[i, 2].set_title(f"Predicted Mask {idx}\nF1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and print overall metrics
    overall_f1 /= num_samples
    overall_precision /= num_samples
    overall_recall /= num_samples
    
    print(f"Overall Metrics (average of {num_samples} samples):")
    print(f"F1 Score: {overall_f1:.4f}")
    print(f"Precision: {overall_precision:.4f}")
    print(f"Recall: {overall_recall:.4f}")



In [26]:
import os

base_path = "../data/raw/H&E"

print("Contents of base_path:")
print(os.listdir(base_path))

for fold in ["Fold 1", "Fold 2", "Fold 3"]:
    fold_path = os.path.join(base_path, fold)
    if os.path.exists(fold_path):
        print(f"\nContents of {fold_path}:")
        print(os.listdir(fold_path))
    else:
        print(f"\n{fold_path} does not exist")
        

model = trained_model  # Your trained CellViT model
test_dataset = CellSegmentationDataset(
    data_splits[chosen_split]['test']['images'],
    data_splits[chosen_split]['test']['masks'],
    image_transform=image_transform,
    mask_transform=mask_transform
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_test_images = len(test_dataset)
print(num_test_images)
# Visualize and calculate metrics
visualize_prediction_with_metrics(model, test_dataset, device, num_samples=3)

Contents of base_path:
['._.DS_Store', '.DS_Store', 'Fold 2', '._Fold 1', 'Fold 1', 'Fold 3']

Contents of ../data/raw/H&E/Fold 1:
['._images', '._.DS_Store', '._README.md', '._masks', 'images', '.DS_Store', 'README.md', 'masks']

Contents of ../data/raw/H&E/Fold 2:
['._.DS_Store', '._README.md', 'images', '.DS_Store', 'README.md', 'masks']

Contents of ../data/raw/H&E/Fold 3:
['._.DS_Store', '._README.md', 'images', '.DS_Store', 'README.md', 'masks']


NameError: name 'data_splits' is not defined

In [21]:
!pwd

/rsrch5/home/plm/yshokrollahi/vitamin-p-new/notebooks
